<a href="https://colab.research.google.com/github/SaiGanesh26/Gender-Classification-Age-Prediction/blob/master/AgeGender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Age Prediction & Gender Classification

In [1]:
#!pip install cvlib

You should consider upgrading via the 'c:\users\vemul\appdata\local\programs\python\python36\python.exe -m pip install --upgrade pip' command.


In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cvlib as cv
import tensorflow as tf
from tensorflow.keras.models import load_model
# from google.colab import  files
# from google.colab.patches import cv2_imshow

#### Load the required models

In [6]:
gen_model = load_model("models/batch_norm_rms_97.5acc_50epo")
age_model = load_model("models/agemodel_CNN2.h5")

## For Existing video

#### Video Input
* If existing video file is to be used then file path is given
* If live web cam is used just enter 0

In [4]:
def detect_face_video(img):
    label_dict={0:'Female',1:'Male'}
    color_dict={0:(0,255,0),1:(0,0,255)}
    age_dict ={0: '10-22', 1:'23-26', 2:'27-31', 3:'32-39', 4:'40-51', 5:'52-99'}

    faces, confidences = cv.detect_face(img) 
    for index,face in enumerate(faces):
        x1,y1 = face[0],face[1]
        x2,y2 = face[2],face[3]
        image = img[y1-20:y2+20,x1-20:x2+20]
        image_gry = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        resize_img = cv2.resize(image_gry,(200,200))
        norm_img = resize_img/255.0
        reshaped=np.reshape(norm_img,(1,200,200,1))
        #reshaped = np.vstack([reshaped])
        
        gender_res = gen_model.predict(reshaped) # gender prediction
        age_result = age_model.predict(reshaped) # age prediction

        if(gender_res[0][0]<0.5):
            label=0
        else:
            label=1

        cv2.rectangle(img,(x1,y1),(x2,y2),color_dict[label],thickness = 5)
        # cv2.rectangle(img,(x2,y2),(x2,y2),color_dict[label],-1)
        cv2.putText(img, label_dict[label]+ ', '+ age_dict[np.argmax(age_result)], (x1, y1+15),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),4)
    return img
        

In [2]:
video = cv2.VideoCapture('test1.mp4')  # Sincle colab has no option for live recording, existing file is used

In [3]:
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('test5_out.mp4',cv2.VideoWriter_fourcc(*'XVID'),25,(width,height))

* colab gives the output of video files as a series of each frame present in video file

In [ ]:
if video.isOpened() == False:
  print('Error! File Not found')
while video.isOpened():
    ret,frame = video.read()
    if ret == True:
        frame = detect_face_video(frame)  #detect the face for each frame displayed on video
        out.write(frame)
    if cv2.waitKey(27) & 0xFF == ord('q'): #if live webcam is given input then video can be quit using letter 'q'
        break
    else:
        break
video.release()
out.release()
cv2.destroyAllWindows()

## For image

In [9]:
def detect_face_image(img):
    label_dict={0:'Female',1:'Male'}
    color_dict={0:(0,255,0),1:(0,0,255)}
    age_dict ={0: '10-22', 1:'23-26', 2:'27-31', 3:'32-39', 4:'40-51', 5:'52-99'}

    faces, confidences = cv.detect_face(img) 

    for index,face in enumerate(faces):
        x1,y1 = face[0],face[1]
        x2,y2 = face[2],face[3]
        image = img[y1-20:y2+20,x1-20:x2+20]
        image_gry = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        resize_img = cv2.resize(image_gry,(200,200))
        norm_img = resize_img/255.0
        reshaped=np.reshape(norm_img,(1,200,200,1))
        #reshaped = np.vstack([reshaped])
        
        gender_res = gen_model.predict(reshaped)
        age_result = age_model.predict(reshaped)

        if(gender_res[0][0]<0.5):
            label=0
        else:
            label=1

        cv2.rectangle(img,(x1,y1),(x2,y2),color_dict[label],thickness = 5)
        # cv2.rectangle(img,(x2,y2),(x2,y2),color_dict[label],-1)
        cv2.putText(img, label_dict[label]+ ', '+ age_dict[np.argmax(age_result)], (x1, y1+15),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),3)
    return img
        

In [14]:
image = cv2.imread("temp1.JPG")
out_image = detect_face_image(image)
plt.imshow(out_image)

## For live Webcam Video

In [5]:
webcam = cv2.VideoCapture(0)
while True:
    ret,frame = webcam.read()
    frame = detect_face_video(frame)  #detect the face for each frame displayed on video
    cv2.imshow("gender detection", frame)
    k = cv2.waitKey(30) & 0xff
    if k==27:
        break
webcam.release()
cv2.destroyAllWindows()